## Single Nuron Neural Network as a "Hello World!" application.

Verify the TensorFlow Version

In [1]:
import tensorflow as tf;
print(tf.__version__)

2.13.0


Building a Neural Netowrk using TensorFlow and Keras APIs

In [ ]:
from keras.src.engine.training import optimizer
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))

The same code with it showing the weights and loss.

In [ ]:
from keras.src.utils.data_utils import SequenceEnqueuer
from keras.src.engine.training import optimizer

import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))
print("Here is what I learned: {}".format(l0.get_weights()))

After running this code, it will you what it thinks the X is, and then what it thinks the relationship of Y to X.

## Chapter 2
### Computer Vision

In this tutorial, we will make a neural network that will look at an image of a piece of clothing, then categorize it and try to guess the correct category.

#### Neurons for Vision

In Chapter 1, you saw a very simple secenario where a machine was given a set of X and Y values, and it learned that the relationship between was Y=2X-1. This was done using a very simple neural network with one layer and one neuron.  

In this example we are uisng Fashion-MNIST dataset that comes with Keras. It is a set of images of clothing to train the model on. Each of our images is a set of 784 values (28 x 28) between 0 and 255. They can be our X.

We know that we have 10 different types of images in our dataset, so let's consider them to be our Y. Now we want to learn what the function looks like where Y is a function of X.

Below is the code for the model including the dataset.

In [24]:
import tensorflow as tf
data = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=25)
#model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)
print(classifications[9])
print(test_labels[9])

Epoch 1/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.5001 - accuracy: 0.8239
Epoch 2/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3813 - accuracy: 0.8641
Epoch 3/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3404 - accuracy: 0.8771
Epoch 4/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3169 - accuracy: 0.8825
Epoch 5/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2964 - accuracy: 0.8916
Epoch 6/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2820 - accuracy: 0.8968
Epoch 7/25
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2683 - accuracy: 0.8992
Epoch 8/25
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2564 - accuracy: 0.9044
Epoch 9/25
1875/1875 [==============================] - 10s 6ms/step - loss: 0.2478 - accuracy: 0.9079
Epoch 10/25
1875/1875 [==============================] - 8s 4ms/step - loss: 

Here we can pick and choose which image in the test-images array to run through the neural network by using the

```
classifications = model.predict(test_images)  - loads the test images into the model.predict function
print(classifications['array-number']) - prints the softmax 10 neuron weights
print(test_label['matching-array-nunber']) - actual classification number
```



code.

The number in both array values must be the same to choose the same test image vs the same fashion category (0-9).

In [ ]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

Play with the values of the classifications array and the test_labels array to discover different chances the neural network had a chance to guess the clothing type. You can see the weights of all 10 final neurons and then the actual type number (0-9).

You can see how accurate it is based off the number of times you train the network in the section above.